In [1]:
import math
import tkinter as tk
from tkinter import ttk

In [2]:
def create_table(canvas):
    tree = ttk.Treeview(canvas,show="headings")
    tree["columns"] = ("Enunciado", "id", "Valores")
    tree.heading("Enunciado", text="Enunciado")
    tree.heading("id", text="id")
    tree.heading("Valores", text="Valores")
    for i in range(50):
        tree.insert("", "end", values=("Exercício", i, "física"))
    tree.grid(row=0, column=0)
    tree.place(x=canvas.winfo_width(), y=canvas.winfo_height())

In [3]:
def resize_canvas(event):
    global color1, color2
    
    canvas.delete("all")
    
    color1, color2 = set_color()
    
    create_gradient_background(0, 0, event.width, event.height, color1, color2)

In [4]:
def create_gradient_background(x1, y1, x2, y2, color1, color2):
    # Direção do gradiente
    dx = x2 - x1
    dy = y2 - y1
    
    #Cálculo para interpolação das cores
    steps = max(dx, dy)
    step_r = (color2[0] - color1[0]) / steps
    step_g = (color2[1] - color1[1]) / steps
    step_b = (color2[2] - color1[2]) / steps
    
    
    
    for y in range(y1, y2):
        # Calculando cor
        r = int((color1[0] * (y2 - y) + color2[0] * (y - y1)) / (y2 - y1))
        g = int((color1[1] * (y2 - y) + color2[1] * (y - y1)) / (y2 - y1))
        b = int((color1[2] * (y2 - y) + color2[2] * (y - y1)) / (y2 - y1))
        hex_color = f"#{r:02X}{g:02X}{b:02X}"
        
        canvas.create_line(x1, y, x2, y, fill=hex_color)

In [5]:
def set_color():
    c1 = (107, 31, 124)  # Rosa #A1405D
    c2 = (161, 64, 93)  # Roxo #6B1F7C
    return c1, c2

In [6]:
def create_canvas(root, w, h):
    global canvas
    canvas = ctk.Canvas(root, width=w, height=h)
    canvas.pack(fill=tk.BOTH, expand=True)
    canvas.bind("<Configure>", resize_canvas)
    create_gradient_background(0, 0, w, h, color1, color2)

In [7]:
def start_interface(root, w, h):
    global canvas, color1, color2
    canvas = tk.Canvas(root, width=w, height=h, bg="#A1405D", highlightthickness=0)
    canvas.pack(fill=tk.BOTH, expand=True)
    canvas.bind("<Configure>", resize_canvas)
    color1, color2 = set_color()
    create_gradient_background(0, 0, w, h, color1, color2)
    create_table(canvas)

In [8]:
def awake_interface():
    root = tk.Tk()
    root.geometry("1280x780")
    root.title("Main Tab")
    height = 780
    width = 1280
    start_interface(root, width, height)
    root.mainloop()

In [9]:
awake_interface()